In [27]:
import pandas as pd
import numpy as np
from exerpy.analyses import _load_json
from exerpy import ExergyAnalysis

In [28]:
def diff_between_simulators(testcase):
    simulator_results = []
    for path in testcase.values():
        contents = _load_json(path)
        if "settings" not in contents:
            contents["settings"] = {}

        simulator_results += [
            ExergyAnalysis.from_json(path, **contents["settings"])
        ]

    sim1 = simulator_results[0]
    sim2 = simulator_results[1]

    columns = ["m", "p", "T"]
    if sim1.chemExLib is not None:
        columns.append("e_CH")
    if sim1.split_physical_exergy:
        columns.append("e_M")
        columns.append("e_T")
    else:
        columns.append("e_PH")

    df_sim1 = pd.DataFrame.from_dict(
        sim1._connection_data, orient="index"
    ).sort_index()[columns].dropna(how="all")
    df_sim2 = pd.DataFrame.from_dict(
        sim2._connection_data, orient="index"
    ).sort_index()[columns].dropna(how="all")

    overlapping_index = list(
        set(df_sim1.index.tolist()) & set(df_sim2.index.tolist())
    )
    df_sim1 = df_sim1.loc[overlapping_index].round(6)
    df_sim2 = df_sim2.loc[overlapping_index].round(6)

    # inf means that sim2 has 0 value, comparison does not make sense there
    # and sometimes there seem to be NaN values in the dataframes, those are
    # removed as well
    return (
        (df_sim1 - df_sim2) / df_sim2
    ).abs().replace(np.inf, 0).fillna(0)

In [29]:
testcase = {
    "ebsilon": "hp_ebs.json",
    "tespy": "hp_tespy.json"
}
diff_between_simulators(testcase)


,m,p,T,e_PH
11,0.000002,0.000000e+00,0.000000e+00,0.000000
21,0.000000,0.000000e+00,0.000000e+00,0.000009
23,0.000000,0.000000e+00,0.000000e+00,0.000003
32,0.000079,0.000000e+00,3.625946e-09,0.000145
22,0.000000,0.000000e+00,0.000000e+00,0.000009
34,0.000079,0.000000e+00,3.992982e-05,0.000062
30,0.000079,0.000000e+00,0.000000e+00,0.000126
12,0.000002,9.971237e-10,1.134148e-05,0.000227
13,0.000002,0.000000e+00,0.000000e+00,0.000266
33,0.000079,1.454546e-10,1.154332e-05,0.000003


In [30]:
testcase = {
    "tespy": "hp_tespy.json",
    "aspen": "hp_aspen.json"
}
diff_between_simulators(testcase)

,m,p,T,e_PH
11,0.000483,0.0,0.000000e+00,0.000000e+00
21,0.000000,0.0,0.000000e+00,6.677584e-07
23,0.000000,0.0,2.543558e-09,6.815941e-07
32,0.000093,0.0,2.538162e-08,1.529795e-04
22,0.000000,0.0,0.000000e+00,6.665134e-07
34,0.000093,0.0,4.539902e-05,7.303921e-05
12,0.000483,0.0,9.652330e-07,2.210114e-04
13,0.000483,0.0,0.000000e+00,6.316969e-03
33,0.000093,0.0,1.231574e-06,7.624343e-06
31,0.000093,0.0,0.000000e+00,1.327337e-04


In [31]:
testcase = {
    "ebsilon": "hp_ebs.json",
    "aspen": "hp_aspen.json"
}
diff_between_simulators(testcase)

,m,p,T,e_PH
11,0.000485,0.000000e+00,0.000000e+00,0.000000
21,0.000000,0.000000e+00,0.000000e+00,0.000009
23,0.000000,0.000000e+00,2.543558e-09,0.000002
32,0.000013,0.000000e+00,2.900757e-08,0.000008
22,0.000000,0.000000e+00,0.000000e+00,0.000009
34,0.000013,0.000000e+00,5.471011e-06,0.000011
12,0.000485,9.971237e-10,1.037626e-05,0.000006
13,0.000485,0.000000e+00,0.000000e+00,0.006581
33,0.000013,1.454546e-10,1.031176e-05,0.000005
31,0.000013,0.000000e+00,0.000000e+00,0.000007


In [32]:
# Load the CSV files
tespy_results = pd.read_csv("hp_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("hp_components_ebsilon.csv", index_col=0)

tespy_results

# Step 1: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 2: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 3: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns].copy()
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,COMP,125.814542,125.819130,4.587055e-03,0.003646
1,COND,91.493965,91.482067,-1.189737e-02,-0.013005
2,EVA,647.197052,647.061438,-1.356148e-01,-0.020959
3,FAN,104.880549,104.855562,-2.498649e-02,-0.023829
4,MOT1,10.744765,10.742327,-2.438023e-03,-0.022695
5,MOT2,13.614950,13.615297,3.478135e-04,0.002555
6,MOT3,0.000974,0.000973,-4.130147e-07,-0.042437
7,PUMP,0.010569,0.010547,-2.236978e-05,-0.212096
8,VAL,144.574924,144.605102,3.017818e-02,0.020869
9,TOT,1138.332290,1138.192444,-1.398468e-01,-0.012287
